In [ ]:
!pip install tf-keras

In [ ]:
import os
import requests
import logging

import gradio as gr
from pymongo import MongoClient
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import torch

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Connect to Qdrant (adjust host/port if different)
qdrant_client = QdrantClient(host='rag_qdrant', port=6333)
collection_name = 'rag_collection'

# Embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


In [2]:
import json
def retrieve_relevant_chunks(query_embedding, top_k=5):
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=top_k
    )
    retrieved_texts = [result.payload['chunk'] for result in search_results]
    return retrieved_texts

def call_ollama(prompt, model="llama2"):
    payload = {
        "model": model,
        "prompt": prompt,
        "options": {
            "stream": False  # Request a full response, not streaming
        }
    }

    url = os.getenv("OLLAMA_HOST", "http://host.docker.internal:11434")
    try:
        # Send the POST request
        response = requests.post(f"{url}/api/generate", json=payload, timeout=30)  # Add timeout
        response.raise_for_status()

        # Handle raw response in case of multiple JSON objects
        raw_response = response.text.strip()  # Raw response as text
        # print("Raw Response:", raw_response)

        # Combine 'response' fields from multiple JSON objects
        responses = []
        for line in raw_response.splitlines():
            try:
                data = json.loads(line)
                if 'response' in data:
                    responses.append(data['response'])
            except json.JSONDecodeError as e:
                logger.error(f"Error parsing line as JSON: {e} - Line: {line}")

        # Join all parts of the response
        full_response = ''.join(responses)
        return full_response if full_response else "No response generated."
    except requests.exceptions.RequestException as e:
        logger.error(f"Error calling Ollama API: {e}")
        return "Sorry, I'm having trouble connecting to the language model."


In [3]:
def generate_response(query):
    if not query.strip():
        return "Please enter a query."
    
    # Embed the user query
    query_embedding = embedding_model.encode(query)

    # Retrieve relevant chunks from Qdrant
    retrieved_texts = retrieve_relevant_chunks(query_embedding, top_k=5)

    # Construct the prompt
    context = "\n".join(retrieved_texts)
    prompt = f"You are a helpful ROS2 expert assistant. Use the following context to answer the question. If you cannot find the answer in the context, say so. If providing code, ensure it's well-commented. Context:\n{context}\n\nQuestion:\n{query}\n\nAnswer:"

    answer = call_ollama(prompt, model="llama2").strip()
    if not answer:
        answer = "I'm not sure how to answer that."
    return answer


In [4]:
test_query = "How do I navigate to a specific pose in ROS2? Can you provide me with code for this task?"
print("\n Answer:", generate_response(test_query))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://rag_qdrant:6333/collections/rag_collection/points/search "HTTP/1.1 200 OK"



 Answer: To navigate to a specific pose in ROS2 using the `navigation` stack, you can use the `NavigateToPoseAction` action server. Here's an example code snippet that demonstrates how to use this action server:
```python
import rospy
from geometry_msgs.msg import PoseStamped

# Initialize ROS node
rospy.init_node('navigate_to_pose')

# Set the goal pose
goal = PoseStamped()
goal.header.stamp = rospy.Time(10)  # Set the timestamp to a random value
goal.position.x = 0.5
goal.position.y = 0.5
goal.position.z = 0.5
goal.orientation = 1.0

# Create an action server for navigating to the goal pose
server = rospy.ActionServer('navigate_to_pose', ' navigation2::NavigateToPoseAction')

try:
    # Subscribe to the goal pose topic
    rospy.Subscriber('/goal_pose', PoseStamped, server)
    
    # Wait for a goal pose message
    while True:
        try:
            goal_msg = rospy.wait_for_message('/goal_pose', PoseStamped)
            if goal_msg:
                server.execute(goal_msg)
    

In [5]:
import gradio as gr

iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs="text",
    title="ROS2 RAG Assistant",
    description="Ask questions about ROS2. The system retrieves relevant info and uses the saavysingh/llama2-rag model to generate an answer."
)

# Launch the Gradio interface
iface.launch(server_name="0.0.0.0", server_port=7860, share=False)


INFO:httpx:HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
INFO:httpx:HTTP Request: GET http://localhost:7860/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://localhost:7860/ "HTTP/1.1 200 OK"


Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
